# Import Dependencies

In [1]:
# !pip install -U googlemaps

In [ ]:
# Dependencies and Setup FOR API
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import requests
import time
import datetime as dt
import json

import googlemaps
from api_keys import g_key

## Read and Store Data Science Jobs Salaries CSV file into DataFrame

In [36]:
# Read and Store linkedin_jobs_usa file into dataframe 
jobs_stats = "data/data_science_jobs_salaries.csv"
jobs_stats_df = pd.read_csv(jobs_stats)
jobs_stats_df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2021e,EN,FT,Data Science Consultant,54000,EUR,64369,DE,50,DE,L
1,2020,SE,FT,Data Scientist,60000,EUR,68428,GR,100,US,L
2,2021e,EX,FT,Head of Data Science,85000,USD,85000,RU,0,RU,M
3,2021e,EX,FT,Head of Data,230000,USD,230000,RU,50,RU,L
4,2021e,EN,FT,Machine Learning Engineer,125000,USD,125000,US,100,US,S


## Transform the Dataframe

In [26]:
# Check the null values (if any) in the df
jobs_stats_df.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [42]:
# Fix the work_year column - remove 'e'
jobs_stats_df['work_year'] = jobs_stats_df['work_year'].str.strip('e')
jobs_stats_df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2021,EN,FT,Data Science Consultant,54000,EUR,64369,DE,50,DE,L
1,2020,SE,FT,Data Scientist,60000,EUR,68428,GR,100,US,L
2,2021,EX,FT,Head of Data Science,85000,USD,85000,RU,0,RU,M
3,2021,EX,FT,Head of Data,230000,USD,230000,RU,50,RU,L
4,2021,EN,FT,Machine Learning Engineer,125000,USD,125000,US,100,US,S


In [49]:
# Check the unique job titles in the df
print(jobs_stats_df['job_title'].nunique())
print(jobs_stats_df['job_title'].unique())

43
['Data Science Consultant' 'Data Scientist' 'Head of Data Science'
 'Head of Data' 'Machine Learning Engineer' 'Data Analytics Manager'
 'Research Scientist' 'Data Analyst' 'Data Engineer'
 'Data Science Engineer' 'Manager Data Science' 'Data Engineering Manager'
 'AI Scientist' '3D Computer Vision Researcher' 'ML Engineer'
 'Applied Data Scientist' 'Director of Data Science'
 'Data Analytics Engineer' 'Lead Data Analyst' 'BI Data Analyst'
 'Machine Learning Scientist' 'Machine Learning Infrastructure Engineer'
 'Marketing Data Analyst' 'Lead Data Engineer'
 'Director of Data Engineering' 'Computer Vision Software Engineer'
 'Financial Data Analyst' 'Cloud Data Engineer' 'Principal Data Analyst'
 'Big Data Architect' 'Lead Data Scientist' 'Data Architect'
 'Big Data Engineer' 'Staff Data Scientist' 'Data Science Manager'
 'Business Data Analyst' 'Principal Data Scientist'
 'Computer Vision Engineer' 'Product Data Analyst'
 'Principal Data Engineer' 'Finance Data Analyst'
 'Applied M

In [51]:
# Check the unique countries in the df
print(jobs_stats_df['company_location'].nunique())
print(jobs_stats_df['company_location'].unique())

41
['DE' 'US' 'RU' 'FR' 'AT' 'CA' 'UA' 'NG' 'IN' 'ES' 'PL' 'GB' 'PT' 'DK'
 'SG' 'MX' 'TR' 'NL' 'AE' 'JP' 'CN' 'HU' 'KE' 'CO' 'NZ' 'IR' 'CL' 'PK'
 'BE' 'GR' 'SI' 'BR' 'CH' 'IT' 'MD' 'LU' 'VN' 'AS' 'HR' 'IL' 'MT']


In [59]:
# Store the df in JSON format for dashboard creation
jobs_stats_df.to_json('stats.json', orient = 'records', compression = 'infer', index = 'true', indent=2)
 

## Read and Store USA jobs CSV file into DataFrame

In [27]:
# Read and Store linkedin_jobs_usa file into dataframe 
jobs_usa = "data/linkedin_jobs_usa.csv"
jobs_usa_df = pd.read_csv(jobs_usa)
jobs_usa_df.head()

,title,company,description,onsite_remote,salary,location,criteria,posted_date,link
0,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,Buffalo-Niagara Falls Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...
1,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"San Jose, CA","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...
2,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Texas, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...
3,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Illinois, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...
4,Entry-Level Data Analyst,The Federal Savings Bank,"The Federal Savings Bank, a national bank and ...",onsite,NaN,"Chicago, IL","[{'Seniority level': 'Entry level'}, {'Employm...",2022-11-17,https://www.linkedin.com/jobs/view/entry-level...


## Transform the DataFrame

### Add coordinates of the locations using Googlemaps

In [12]:
# gmaps = googlemaps.Client(key= g_key)

# jobs_usa_df['latitude'] = None
# jobs_usa_df['longitude'] = None

# for i in range(0, len(jobs_usa_df)):     
#     try:        
        
#         geocode_result = gmaps.geocode(jobs_usa_df.iat[i, 5])
#         lat= geocode_result[0]['geometry']['location']['lat']
#         lng= geocode_result[0]['geometry']['location']['lng']
#         jobs_usa_df.iat[i, jobs_usa_df.columns.get_loc('latitude')] = lat
#         jobs_usa_df.iat[i, jobs_usa_df.columns.get_loc('longitude')] = lng
        
#     except Exception as e:
#         print(e)
        
# jobs_usa_df  
       

list index out of range
list index out of range


,title,company,description,onsite_remote,salary,location,criteria,posted_date,link,latitude,longitude
0,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,Buffalo-Niagara Falls Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,42.851801,-78.747621
1,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"San Jose, CA","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,37.33874,-121.885253
2,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Texas, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,31.968599,-99.901813
3,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Illinois, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,40.633125,-89.398528
4,Entry-Level Data Analyst,The Federal Savings Bank,"The Federal Savings Bank, a national bank and ...",onsite,NaN,"Chicago, IL","[{'Seniority level': 'Entry level'}, {'Employm...",2022-11-17,https://www.linkedin.com/jobs/view/entry-level...,41.878114,-87.629798
...,...,...,...,...,...,...,...,...,...,...,...
2840,Junior Data Analyst,Iris Software Inc.,"Iris's client, one of the world's largest fina...",hybrid,NaN,"Texas, United States","[{'Seniority level': 'Mid-Senior level'}, {'Em...",2022-10-21,https://www.linkedin.com/jobs/view/junior-data...,31.968599,-99.901813
2841,Data Analyst (SQL),Marwood Group,The Marwood Group (Marwood) is a leading healt...,hybrid,"$75,000.00\r\n -\r\n $95...",New York City Metropolitan Area,"[{'Seniority level': 'Mid-Senior level'}, {'Em...",2022-10-26,https://www.linkedin.com/jobs/view/data-analys...,40.712776,-74.005954
2842,Data Analyst,SmartSense by Digi,"Join a high-performing, tight-knit team at a f...",hybrid,NaN,"Mishawaka, IN","[{'Seniority level': 'Associate'}, {'Employmen...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,41.661993,-86.158616
2843,Data Analyst,Synergy Search,Nashville (Berry Hill) based company looking t...,hybrid,"$85,000.00\r\n -\r\n $95...",Nashville Metropolitan Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-14,https://www.linkedin.com/jobs/view/data-analys...,36.162646,-86.781497


### Save the updated dataframe as csv file 

In [13]:
#  Save the new dataframe as CSV file and use this file for further use so that we are not makining the api call again
# jobs_usa_df.to_csv('data/updated_jobs_usa.csv')

### Read and Store the updated CSV file into DataFrame

In [18]:
# Read and Store updated_jobs_usa file into dataframe 
updated_jobs_usa = "data/updated_jobs_usa.csv"
updated_jobs_usa_df = pd.read_csv(updated_jobs_usa, index_col=0)
updated_jobs_usa_df

,title,company,description,onsite_remote,salary,location,criteria,posted_date,link,latitude,longitude
0,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,Buffalo-Niagara Falls Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,42.851801,-78.747621
1,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"San Jose, CA","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,37.338740,-121.885253
2,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Texas, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,31.968599,-99.901813
3,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Illinois, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,40.633125,-89.398528
4,Entry-Level Data Analyst,The Federal Savings Bank,"The Federal Savings Bank, a national bank and ...",onsite,NaN,"Chicago, IL","[{'Seniority level': 'Entry level'}, {'Employm...",2022-11-17,https://www.linkedin.com/jobs/view/entry-level...,41.878114,-87.629798
...,...,...,...,...,...,...,...,...,...,...,...
2840,Junior Data Analyst,Iris Software Inc.,"Iris's client, one of the world's largest fina...",hybrid,NaN,"Texas, United States","[{'Seniority level': 'Mid-Senior level'}, {'Em...",2022-10-21,https://www.linkedin.com/jobs/view/junior-data...,31.968599,-99.901813
2841,Data Analyst (SQL),Marwood Group,The Marwood Group (Marwood) is a leading healt...,hybrid,"$75,000.00\r\n -\r\n $95...",New York City Metropolitan Area,"[{'Seniority level': 'Mid-Senior level'}, {'Em...",2022-10-26,https://www.linkedin.com/jobs/view/data-analys...,40.712776,-74.005954
2842,Data Analyst,SmartSense by Digi,"Join a high-performing, tight-knit team at a f...",hybrid,NaN,"Mishawaka, IN","[{'Seniority level': 'Associate'}, {'Employmen...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,41.661993,-86.158616
2843,Data Analyst,Synergy Search,Nashville (Berry Hill) based company looking t...,hybrid,"$85,000.00\r\n -\r\n $95...",Nashville Metropolitan Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-14,https://www.linkedin.com/jobs/view/data-analys...,36.162646,-86.781497


In [19]:
# Remane columns
updated_jobs_usa_df.rename(columns={'onsite_remote':'work_environment'}, inplace = True)

# Drop NaN values
updated_jobs_usa_df = updated_jobs_usa_df.dropna(subset=['latitude', 'longitude'])  

In [20]:
# Convert lattitude and longitude to float types
updated_jobs_usa_df['latitude'] = updated_jobs_usa_df['latitude'].astype(float)
updated_jobs_usa_df['longitude'] = updated_jobs_usa_df['longitude'].astype(float)

# reset index 
updated_jobs_usa_df.reset_index(drop=True)

C:\Users\meena\AppData\Local\Temp\ipykernel_16768\468078232.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_jobs_usa_df['latitude'] = updated_jobs_usa_df['latitude'].astype(float)
C:\Users\meena\AppData\Local\Temp\ipykernel_16768\468078232.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_jobs_usa_df['longitude'] = updated_jobs_usa_df['longitude'].astype(float)


,title,company,description,work_environment,salary,location,criteria,posted_date,link,latitude,longitude
0,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,Buffalo-Niagara Falls Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,42.851801,-78.747621
1,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"San Jose, CA","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,37.338740,-121.885253
2,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Texas, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,31.968599,-99.901813
3,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Illinois, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,40.633125,-89.398528
4,Entry-Level Data Analyst,The Federal Savings Bank,"The Federal Savings Bank, a national bank and ...",onsite,NaN,"Chicago, IL","[{'Seniority level': 'Entry level'}, {'Employm...",2022-11-17,https://www.linkedin.com/jobs/view/entry-level...,41.878114,-87.629798
...,...,...,...,...,...,...,...,...,...,...,...
2838,Junior Data Analyst,Iris Software Inc.,"Iris's client, one of the world's largest fina...",hybrid,NaN,"Texas, United States","[{'Seniority level': 'Mid-Senior level'}, {'Em...",2022-10-21,https://www.linkedin.com/jobs/view/junior-data...,31.968599,-99.901813
2839,Data Analyst (SQL),Marwood Group,The Marwood Group (Marwood) is a leading healt...,hybrid,"$75,000.00\r\n -\r\n $95...",New York City Metropolitan Area,"[{'Seniority level': 'Mid-Senior level'}, {'Em...",2022-10-26,https://www.linkedin.com/jobs/view/data-analys...,40.712776,-74.005954
2840,Data Analyst,SmartSense by Digi,"Join a high-performing, tight-knit team at a f...",hybrid,NaN,"Mishawaka, IN","[{'Seniority level': 'Associate'}, {'Employmen...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,41.661993,-86.158616
2841,Data Analyst,Synergy Search,Nashville (Berry Hill) based company looking t...,hybrid,"$85,000.00\r\n -\r\n $95...",Nashville Metropolitan Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-14,https://www.linkedin.com/jobs/view/data-analys...,36.162646,-86.781497


## Convert the Dataframe into Geojson and save as .js file

In [21]:
# Create a function to convert the dataframe into geojson
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    
    # Create a new python dict to contain the geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # Loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        
        # Create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # Fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # For each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # Add this feature to the list of features inside the dict
        geojson['features'].append(feature)
    
    return geojson

In [22]:
# Pass in the dataFrame, list of columns to to the above function
cols = ['title', 'company', 'description', 'work_environment', 'salary', 'location', 'criteria', 'posted_date', 'link']
geojson = df_to_geojson(updated_jobs_usa_df, cols)
#geojson

In [23]:
# Save it to a .js file for dashboard creation
output_filename = 'jobs_usa.js'
with open(output_filename, 'wt') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 


## Read and Store Canada and Africa jobs CSV files into DataFrames

In [ ]:
# Read and Store linkedin_jobs_canada file into dataframe 
jobs_canada = "Data/linkedin_jobs_canada.csv"
jobs_canada_df = pd.read_csv(jobs_canada)
#jobs_canada_df


# Read and Store linkedin_jobs_africa file into dataframe 
jobs_africa = "Data/linkedin_jobs_africa.csv"
jobs_africa_df = pd.read_csv(jobs_africa)
#jobs_africa_df

## # Load the Transformed Dataframe

In [ ]:
# # Connect to local database
# protocol = 'postgresql'
# username = 'postgres'
# password = 'postgres'
# host = 'localhost'
# port = 5432
# database_name = 'Entertainment_db'
# rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
# engine = create_engine(rds_connection_string)

# # Check for tables
# engine.table_names()

# # Use pandas to load csv converted DataFrame into database
# popular_entertainment_df.to_sql(name='popular_entertainment', con=engine, if_exists='append', index=False)

# # Confirm data has been added by querying the popular_entertainment table
# pd.read_sql_query('select * from popular_entertainment', con=engine).head()